In [ ]:
import pandas as pd
import os

In [ ]:
file1 = pd.read_excel("complete_P_threshold_protein_group_FDR_0.01.xlsx")

In [ ]:
file1.head()

In [ ]:
file1['filename'].tolist()

interact
S  /  HP  / H  / 

R2'
'.ipro.pep.xml'

In [ ]:
total_d = {}

for filename in os.listdir("excels"):
    
    #filename = "interact-20190923-Imp-homogenizer-R1.ipro.prot.xls"


    df = pd.read_csv('excels/'+filename, sep='\t')


    for e in range(len(file1['filename'].tolist())):
        f = file1['filename'].tolist()[e]
        
        if type(f) == str:
            if f.split('.')[0] == filename.split('.')[0]:
                #print(f)
                limit = file1['limit'][e]
                #print(file1['limit'][e])
                #print(filename.split('.')[0][:-3])
                experiment_case_name = filename.split('.')[0][:-3]
                experiment_replica_number = int(filename.split('.')[0][-1])
                #print(experiment_replica_number)


    filtrado = df.loc[df['protein probability'] > limit]
    
    #print("{} ------   {}".format(filename, len(filtrado)))
    
    #filtro2 = filtrado.loc[filtrado['group probability'] == filtrado['protein probability']]


    new_df = filtrado[['protein', 'description']].copy()

    final_df = new_df.drop_duplicates(subset ="protein")
    
    print("{} ------   {}".format(filename, len(final_df)))
    
    
    if experiment_case_name not in total_d:
        total_d[experiment_case_name] = {}
    
    total_d[experiment_case_name][experiment_replica_number] = final_df
    print(experiment_replica_number)
    #final_df.to_excel("aquitienes.xls", index=False)
    #final_df.head()

#procesar replicas

overlap_d = {}

experiment_count = 0

for experiment_case in total_d:
    
    experiment_count += 1
    print("Analysing :{}".format(experiment_case))
    print("Case number :{}".format(experiment_count))
    experiment_proteins = []
    protein_dict = {}
    for replica_number in total_d[experiment_case]:
        print("\n    Checking replica number: {}".format(replica_number))
        df_replica = total_d[experiment_case][replica_number]
        proteins = df_replica['protein'].tolist()
        for protein in proteins:
            if protein in protein_dict:
                protein_dict[protein] += 1
            else:
                protein_dict[protein] = 1
        
    overlapping_proteins = []
    for prot in protein_dict:
        if protein_dict[prot] >=3:
            overlapping_proteins.append(prot)

    print("Overlapping proteins found: {} \n\n".format(len(overlapping_proteins)))
    #print(overlapping_proteins)


    overlap_protein_descriptions = {}
    for protein in overlapping_proteins:
        description = df_replica.set_index('protein').loc[protein].description
        overlap_protein_descriptions[protein] = description
        #print(description)

    overlap_d[experiment_case] = overlap_protein_descriptions
    

        



In [ ]:
meat_dict = {}

for case_name in overlap_d:
    d = overlap_d[case_name]
    method = case_name.split('-')[1]  # method sera 'H'
    meat = case_name.split('-')[2]  # meat sera 'beef'
    
    if meat not in meat_dict:
        meat_dict[meat] = {}
    
    meat_dict[meat][method] = d
    

proteins_in_each_meat = {}

overlap_proteins = {}

for meat in meat_dict:
    
    proteins_in_each_meat[meat] = {}
    overlap_proteins[meat] = {}
    
    protein_count = {}
    for method in meat_dict[meat]:
        for protein in meat_dict[meat][method]:
            if protein not in protein_count:
                protein_count[protein] = 0
            protein_count[protein] += 1

    for protein in protein_count:
        if protein_count[protein] >= 3:
            overlap_proteins[meat][protein] = meat_dict[meat]['H'][protein]
                
        

In [ ]:
for meat in overlap_proteins:
    df_result = pd.DataFrame(list(overlap_proteins[meat].items()), columns=['Protein', 'Description'])
    df_result.to_excel(meat+"-proteins.xls", index=False)